# Assignment 02 - Part 2: Pinecone Indexing & Embeddings

## 목표
- BGE-M3 임베딩 모델 로드
- Pinecone 인덱스 생성
- 청크 데이터 임베딩 및 업로드
- BM25 인덱스 생성 및 저장

In [1]:
# 필요한 라이브러리 임포트
import os
import json
import yaml
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from tqdm.auto import tqdm
from typing import List, Dict, Any
import time

# Pinecone
from pinecone import Pinecone, ServerlessSpec

# Embeddings
from sentence_transformers import SentenceTransformer

# BM25
from rank_bm25 import BM25Okapi

# 환경 변수
from dotenv import load_dotenv
load_dotenv()

# 경로 설정
PROJECT_ROOT = Path('/home/dhc99/ajou-llmops-2025-2nd-semester/assignment02')
DATA_DIR = PROJECT_ROOT / 'datasets'
CONFIG_DIR = PROJECT_ROOT / 'configs'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
ARTIFACTS_DIR.mkdir(exist_ok=True)

print(f"✅ Project initialized")
print(f"   Data: {DATA_DIR}")
print(f"   Artifacts: {ARTIFACTS_DIR}")

/home/dhc99/anaconda3/envs/torch_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Project initialized
   Data: /home/dhc99/ajou-llmops-2025-2nd-semester/assignment02/datasets
   Artifacts: /home/dhc99/ajou-llmops-2025-2nd-semester/assignment02/artifacts


In [2]:
# 설정 로드
with open(CONFIG_DIR / 'models.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print("📋 Configuration:")
print(f"  - Embedding Model: {config['embedding']['model_name']}")
print(f"  - Pinecone Index: {config['pinecone']['index_name']}")
print(f"  - Vector Dimension: {config['pinecone']['dimension']}")
print(f"  - Metric: {config['pinecone']['metric']}")

📋 Configuration:
  - Embedding Model: BAAI/bge-m3
  - Pinecone Index: rag-assignment-korquad
  - Vector Dimension: 1024
  - Metric: cosine


In [3]:
# 청크 데이터 로드
chunks_df = pd.read_parquet(DATA_DIR / 'corpus_chunks.parquet')
print(f"✅ Loaded {len(chunks_df):,} chunks")
print(f"\n📊 Sample chunk:")
print(chunks_df.iloc[0][['chunk_id', 'text', 'title', 'doc_id']])

✅ Loaded 641 chunks

📊 Sample chunk:
chunk_id                                  6566495-0-0_chunk_0
text        1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...
title                                                 파우스트_서곡
doc_id                                            6566495-0-0
Name: 0, dtype: object


## 1. 임베딩 모델 로드

In [5]:
# BGE-M3 모델 로드 (safetensors 사용)
print("📥 Loading embedding model...")

# 방법 1: safetensors 사용 (가장 안전)
try:
    print("   Trying with safetensors...")
    embedding_model = SentenceTransformer(
        config['embedding']['model_name'],
        device='cuda' if config['embedding']['device'] == 'cuda' else 'cpu',
        model_kwargs={'use_safetensors': True}  # safetensors 강제 사용
    )
    print("   ✅ Loaded with safetensors!")
except Exception as e:
    print(f"   ⚠️  Safetensors failed: {e}")
    
    # 방법 2: 대체 모델 사용 (multilingual-e5)
    try:
        print("   Trying alternative model: intfloat/multilingual-e5-large...")
        embedding_model = SentenceTransformer(
            "intfloat/multilingual-e5-large",
            device='cuda' if config['embedding']['device'] == 'cuda' else 'cpu'
        )
        print("   ✅ Loaded multilingual-e5-large!")
        # config 업데이트
        config['embedding']['model_name'] = "intfloat/multilingual-e5-large"
        config['pinecone']['dimension'] = 1024  # e5-large dimension
    except Exception as e2:
        print(f"   ⚠️  Alternative model failed: {e2}")
        
        # 방법 3: 더 작은 모델 (all-MiniLM-L6-v2)
        print("   Trying smaller model: paraphrase-multilingual-MiniLM-L12-v2...")
        embedding_model = SentenceTransformer(
            "paraphrase-multilingual-MiniLM-L12-v2",
            device='cuda' if config['embedding']['device'] == 'cuda' else 'cpu'
        )
        print("   ✅ Loaded paraphrase-multilingual-MiniLM-L12-v2!")
        # config 업데이트
        config['embedding']['model_name'] = "paraphrase-multilingual-MiniLM-L12-v2"
        config['pinecone']['dimension'] = 384  # MiniLM dimension

print(f"\n✅ Model loaded: {config['embedding']['model_name']}")
print(f"   Max sequence length: {embedding_model.max_seq_length}")
print(f"   Embedding dimension: {config['pinecone']['dimension']}")

# 테스트 임베딩
test_text = "이것은 테스트 문장입니다."
test_embedding = embedding_model.encode(test_text, normalize_embeddings=config['embedding']['normalize'])
print(f"\n✅ Test embedding shape: {test_embedding.shape}")
print(f"   Dimension: {len(test_embedding)}")

📥 Loading embedding model...
   Trying with safetensors...
   ⚠️  Safetensors failed: BAAI/bge-m3 does not appear to have a file named model.safetensors or model.safetensors.index.json and thus cannot be loaded with `safetensors`. Please make sure that the model has been saved with `safe_serialization=True` or do not set `use_safetensors=True`.
   Trying alternative model: intfloat/multilingual-e5-large...
   ✅ Loaded multilingual-e5-large!

✅ Model loaded: intfloat/multilingual-e5-large
   Max sequence length: 512
   Embedding dimension: 1024

✅ Test embedding shape: (1024,)
   Dimension: 1024


## 2. Pinecone 초기화 및 인덱스 생성

In [6]:
# Pinecone 초기화
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

if not PINECONE_API_KEY:
    raise ValueError("❌ PINECONE_API_KEY not found in environment variables")

pc = Pinecone(api_key=PINECONE_API_KEY)

# 인덱스 이름
index_name = config['pinecone']['index_name']

# 기존 인덱스 확인
existing_indexes = pc.list_indexes().names()
print(f"📋 Existing indexes: {existing_indexes}")

# 인덱스가 이미 존재하면 삭제 (선택적)
if index_name in existing_indexes:
    print(f"⚠️  Index '{index_name}' already exists.")
    response = input("Delete and recreate? (yes/no): ")
    if response.lower() == 'yes':
        pc.delete_index(index_name)
        print(f"🗑️  Deleted index '{index_name}'")
        time.sleep(5)  # Wait for deletion

# 새 인덱스 생성
if index_name not in pc.list_indexes().names():
    print(f"\n🔧 Creating index '{index_name}'...")
    pc.create_index(
        name=index_name,
        dimension=config['pinecone']['dimension'],
        metric=config['pinecone']['metric'],
        spec=ServerlessSpec(
            cloud=config['pinecone']['cloud'],
            region=config['pinecone']['region']
        )
    )
    print(f"✅ Index '{index_name}' created")
    
    # Wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        print("   Waiting for index to be ready...")
        time.sleep(5)
    print("✅ Index is ready!")

# 인덱스 연결
index = pc.Index(index_name)
print(f"\n📊 Index stats:")
print(index.describe_index_stats())

📋 Existing indexes: ['rag-korquad-demo']

🔧 Creating index 'rag-assignment-korquad'...
✅ Index 'rag-assignment-korquad' created
✅ Index is ready!

📊 Index stats:
{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


## 3. 임베딩 생성 및 Pinecone 업로드

In [7]:
def create_embeddings_batch(texts: List[str], batch_size: int = 32) -> np.ndarray:
    """
    배치 단위로 임베딩 생성
    
    Args:
        texts: 텍스트 리스트
        batch_size: 배치 크기
    
    Returns:
        임베딩 배열
    """
    embeddings = embedding_model.encode(
        texts,
        batch_size=batch_size,
        normalize_embeddings=config['embedding']['normalize'],
        show_progress_bar=True
    )
    return embeddings


# 전체 청크 임베딩 생성
print(f"\n🔄 Creating embeddings for {len(chunks_df):,} chunks...")
all_texts = chunks_df['text'].tolist()
all_embeddings = create_embeddings_batch(
    all_texts,
    batch_size=config['embedding']['batch_size']
)

print(f"✅ Embeddings created: {all_embeddings.shape}")

# DataFrame에 임베딩 추가
chunks_df['embedding'] = list(all_embeddings)


🔄 Creating embeddings for 641 chunks...


Batches: 100%|██████████| 21/21 [00:04<00:00,  4.75it/s]

✅ Embeddings created: (641, 1024)


In [8]:
def upsert_to_pinecone(index, chunks_df: pd.DataFrame, batch_size: int = 100):
    """
    Pinecone에 벡터 업로드
    
    Args:
        index: Pinecone 인덱스
        chunks_df: 청크 데이터프레임
        batch_size: 업로드 배치 크기
    """
    total_chunks = len(chunks_df)
    
    for i in tqdm(range(0, total_chunks, batch_size), desc="Uploading to Pinecone"):
        batch_df = chunks_df.iloc[i:i+batch_size]
        
        # Pinecone 벡터 포맷 생성
        vectors = []
        for _, row in batch_df.iterrows():
            metadata = {
                'text': row['text'][:1000],  # Pinecone metadata limit
                'doc_id': str(row['doc_id']),
                'title': str(row['title'])[:100],
                'language': row['language'],
                'source': row['source'],
                'chunk_index': int(row['chunk_index']),
                'section': str(row['section'])
            }
            
            vectors.append({
                'id': str(row['chunk_id']),
                'values': row['embedding'].tolist(),
                'metadata': metadata
            })
        
        # 업로드
        index.upsert(vectors=vectors)
        time.sleep(0.1)  # Rate limiting
    
    print(f"✅ Uploaded {total_chunks:,} vectors to Pinecone")


# Pinecone에 업로드
print("\n📤 Uploading vectors to Pinecone...")
upsert_to_pinecone(index, chunks_df, batch_size=100)

# 인덱스 통계 확인
time.sleep(5)  # Wait for stats to update
stats = index.describe_index_stats()
print(f"\n📊 Index stats after upload:")
print(f"   Total vectors: {stats['total_vector_count']:,}")
print(f"   Dimension: {stats['dimension']}")


📤 Uploading vectors to Pinecone...


Uploading to Pinecone: 100%|██████████| 7/7 [00:22<00:00,  3.19s/it]


✅ Uploaded 641 vectors to Pinecone

📊 Index stats after upload:
   Total vectors: 641
   Dimension: 1024


## 4. BM25 인덱스 생성

In [17]:
# 텍스트 토크나이제이션 (개선된 한국어 지원)
import re

def tokenize(text: str) -> List[str]:
    """
    한국어 친화적 토크나이저
    - 공백 분리
    - 특수문자 제거
    - N-gram 생성 (2-3 글자 단위)
    """
    # 소문자 변환
    text = text.lower()
    
    # 특수문자 제거 (한글, 영문, 숫자만 유지)
    text = re.sub(r'[^가-힣a-z0-9\s]', ' ', text)
    
    # 공백 기반 토큰
    tokens = text.split()
    
    # 추가: 2-3 글자 n-gram (한국어 검색 향상)
    ngrams = []
    for token in tokens:
        if len(token) >= 2:
            # 2-gram
            for i in range(len(token) - 1):
                ngrams.append(token[i:i+2])
            # 3-gram
            if len(token) >= 3:
                for i in range(len(token) - 2):
                    ngrams.append(token[i:i+3])
    
    # 토큰 + n-gram 결합
    all_tokens = tokens + ngrams
    
    # 중복 제거 및 필터링 (너무 짧은 토큰 제외)
    return [t for t in all_tokens if len(t) >= 1]

print("🔄 Creating BM25 index with improved Korean tokenizer...")

# 모든 청크 토크나이즈
tokenized_corpus = [tokenize(text) for text in tqdm(chunks_df['text'], desc="Tokenizing")]

# BM25 인덱스 생성 (파라미터 조정)
bm25 = BM25Okapi(
    tokenized_corpus,
    k1=1.5,  # Term frequency saturation
    b=0.75   # Length normalization
)

print(f"✅ BM25 index created with {len(tokenized_corpus):,} documents")

# BM25 인덱스 저장
bm25_file = ARTIFACTS_DIR / 'bm25_index.pkl'
with open(bm25_file, 'wb') as f:
    pickle.dump({
        'bm25': bm25,
        'tokenized_corpus': tokenized_corpus,
        'chunk_ids': chunks_df['chunk_id'].tolist()
    }, f)

print(f"✅ BM25 index saved to: {bm25_file}")
print(f"   Size: {bm25_file.stat().st_size / 1024 / 1024:.2f} MB")

🔄 Creating BM25 index with improved Korean tokenizer...


Tokenizing: 100%|██████████| 641/641 [00:00<00:00, 5007.56it/s]


✅ BM25 index created with 641 documents
✅ BM25 index saved to: /home/dhc99/ajou-llmops-2025-2nd-semester/assignment02/artifacts/bm25_index.pkl
   Size: 3.87 MB


## 5. 임베딩 데이터 저장

In [14]:
# 임베딩이 포함된 청크 데이터 저장
# (임베딩은 numpy array이므로 별도 처리)
embeddings_file = ARTIFACTS_DIR / 'embeddings.npy'
np.save(embeddings_file, all_embeddings)
print(f"✅ Embeddings saved to: {embeddings_file}")
print(f"   Shape: {all_embeddings.shape}")
print(f"   Size: {embeddings_file.stat().st_size / 1024 / 1024:.2f} MB")

# 메타데이터만 저장 (임베딩 제외)
chunks_metadata = chunks_df.drop(columns=['embedding']).copy()
metadata_file = ARTIFACTS_DIR / 'chunks_metadata.parquet'
chunks_metadata.to_parquet(metadata_file, index=False)
print(f"\n✅ Chunks metadata saved to: {metadata_file}")

✅ Embeddings saved to: /home/dhc99/ajou-llmops-2025-2nd-semester/assignment02/artifacts/embeddings.npy
   Shape: (641, 1024)
   Size: 2.50 MB

✅ Chunks metadata saved to: /home/dhc99/ajou-llmops-2025-2nd-semester/assignment02/artifacts/chunks_metadata.parquet


## 6. 검색 테스트

In [18]:
# Dense 검색 테스트 (데이터셋에 실제로 있는 질문 사용)
test_query = "대한민국에서 최초로 출시된 스낵은?"
print(f"🔍 Test Query: {test_query}")
print(f"   Expected Answer: 새우깡\n")

# 쿼리 임베딩
query_embedding = embedding_model.encode(
    test_query,
    normalize_embeddings=config['embedding']['normalize']
)

# Pinecone 검색
results = index.query(
    vector=query_embedding.tolist(),
    top_k=5,
    include_metadata=True
)

print("📊 Dense Retrieval Results:")
print("="*80)
for i, match in enumerate(results['matches'], 1):
    print(f"\n{i}. Score: {match['score']:.4f}")
    print(f"   ID: {match['id']}")
    print(f"   Title: {match['metadata'].get('title', 'N/A')}")
    print(f"   Text: {match['metadata']['text'][:200]}...")
    print("-"*80)

🔍 Test Query: 대한민국에서 최초로 출시된 스낵은?
   Expected Answer: 새우깡

📊 Dense Retrieval Results:

1. Score: 0.8713
   ID: 6467462-0-1_chunk_1
   Title: 농심그룹
   Text: '농심'으로 바꾸고 라면시장에 본격적으로 뛰어든다. 이 과정에서 피를 나눈 두 형제는 의절했고, 결국 롯데그룹과 농심그룹으로 갈라지는 계기가 됐다. 1971년 12월, 대한민국 최초의 스낵인 '새우깡'을 출시했다. 짭짤하면서 고소하고 바삭한 식감의 새우깡은 독특한 이름과 함께 시장에서 돌풍을 일으키며 출시 3개월 만에 농심 매출을 2배 가까운 성장을 가져다...
--------------------------------------------------------------------------------

2. Score: 0.8713
   ID: 6484373-0-0_chunk_1
   Title: 농심그룹
   Text: '농심'으로 바꾸고 라면시장에 본격적으로 뛰어든다. 이 과정에서 피를 나눈 두 형제는 의절했고, 결국 롯데그룹과 농심그룹으로 갈라지는 계기가 됐다. 1971년 12월, 대한민국 최초의 스낵인 '새우깡'을 출시했다. 짭짤하면서 고소하고 바삭한 식감의 새우깡은 독특한 이름과 함께 시장에서 돌풍을 일으키며 출시 3개월 만에 농심 매출을 2배 가까운 성장을 가져다...
--------------------------------------------------------------------------------

3. Score: 0.8713
   ID: 6484373-0-1_chunk_1
   Title: 농심그룹
   Text: '농심'으로 바꾸고 라면시장에 본격적으로 뛰어든다. 이 과정에서 피를 나눈 두 형제는 의절했고, 결국 롯데그룹과 농심그룹으로 갈라지는 계기가 됐다. 1971년 12월, 대한민국 최초의 스낵인 '새우깡'을 출시했다. 짭짤하면서 고소하고 바삭한 식감

In [19]:
# BM25 검색 테스트
print(f"\n🔍 BM25 Test Query: {test_query}")
print(f"   Expected Answer: 새우깡\n")

tokenized_query = tokenize(test_query)
print(f"🔤 Tokenized Query: {tokenized_query[:20]}...")  # 처음 20개만 표시

bm25_scores = bm25.get_scores(tokenized_query)

# Top-K 결과
top_k_indices = np.argsort(bm25_scores)[::-1][:5]

print("\n📊 BM25 Retrieval Results:")
print("="*80)
for i, idx in enumerate(top_k_indices, 1):
    chunk = chunks_df.iloc[idx]
    print(f"\n{i}. Score: {bm25_scores[idx]:.4f}")
    print(f"   ID: {chunk['chunk_id']}")
    print(f"   Title: {chunk['title']}")
    print(f"   Text: {chunk['text'][:200]}...")
    print("-"*80)


🔍 BM25 Test Query: 대한민국에서 최초로 출시된 스낵은?
   Expected Answer: 새우깡

🔤 Tokenized Query: ['대한민국에서', '최초로', '출시된', '스낵은', '대한', '한민', '민국', '국에', '에서', '대한민', '한민국', '민국에', '국에서', '최초', '초로', '최초로', '출시', '시된', '출시된', '스낵']...

📊 BM25 Retrieval Results:

1. Score: 28.4991
   ID: 6484386-0-2_chunk_1
   Title: 농심그룹
   Text: '농심'으로 바꾸고 라면시장에 본격적으로 뛰어든다. 이 과정에서 피를 나눈 두 형제는 의절했고, 결국 롯데그룹과 농심그룹으로 갈라지는 계기가 됐다. 1971년 12월, 대한민국 최초의 스낵인 '새우깡'을 출시했다. 짭짤하면서 고소하고 바삭한 식감의 새우깡은 독특한 이름과 함께 시장에서 돌풍을 일으키며 출시 3개월 만에 농심 매출을 2배 가까운 성장을 가져다...
--------------------------------------------------------------------------------

2. Score: 28.4991
   ID: 6484386-0-1_chunk_1
   Title: 농심그룹
   Text: '농심'으로 바꾸고 라면시장에 본격적으로 뛰어든다. 이 과정에서 피를 나눈 두 형제는 의절했고, 결국 롯데그룹과 농심그룹으로 갈라지는 계기가 됐다. 1971년 12월, 대한민국 최초의 스낵인 '새우깡'을 출시했다. 짭짤하면서 고소하고 바삭한 식감의 새우깡은 독특한 이름과 함께 시장에서 돌풍을 일으키며 출시 3개월 만에 농심 매출을 2배 가까운 성장을 가져다...
--------------------------------------------------------------------------------

3. Score: 28.4991
   ID: 6484386-0-0_chun

## 7. Pinecone 프로젝트 공유 안내

**중요: Pinecone 프로젝트에 강사를 초대해야 합니다!**

### 단계:
1. Pinecone 콘솔 접속: https://app.pinecone.io/
2. 프로젝트 설정 → Members
3. 강사 이메일 초대 (권한: Viewer 또는 Editor)
4. 초대 스크린샷 캡처
5. 제출물에 포함

In [ ]:
print("\n" + "="*80)
print("🎉 Indexing Complete!")
print("="*80)
print(f"\n📊 Summary:")
print(f"   Pinecone Index: {index_name}")
print(f"   Total Vectors: {stats['total_vector_count']:,}")
print(f"   Embedding Model: {config['embedding']['model_name']}")
print(f"   BM25 Index: {bm25_file}")
print(f"\n📁 Artifacts:")
print(f"   - {embeddings_file}")
print(f"   - {bm25_file}")
print(f"   - {metadata_file}")
print("\n⚠️  Remember to invite instructor to Pinecone project!")
print("="*80)